# Human Genome Language Model
1. Initialisation
    - 1.1 Imports
    - 1.2 Data processing
2. Tokenisation
    - 2.1 Fixed Length Tokenisation
    - 2.2 Variable Length Tokenisation
3. Experiments
    - 3.1 Comparing fixed and variable length tokenisation strategies

---
# 1. Initialisation
## 1.1 Imports and config
Libraries used include:
- `fastai` for access to the ULMFiT model API
- `pandas` and `matplotlib` for data science toolset

Handy tricks include:
- `autoreload` functionality so that updates to library code are automatically recompiled any time a cell is run.
- `InteractiveShell` is used for multiple outputs from a single cell.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from fastai import *
from fastai.text import *
import sentencepiece as spm
import pandas as pd, matplotlib.pyplot as plt
import numpy as np
# import cupy as np

# Imports from GenomicULMFiT repo
sys.path.append("../../../Genomic-ULMFiT/")
sys.path.append("../..")

from src.processing import process_fasta
from src.config import GRCH38_P13, GENOME
from utils import *

In [ ]:
HUMAN = Path('/home/jovyan/ml_genomics/genomeXL/data/human/')
THESIS = Path('/home/jovyan/ml_genomics/thesis/')

In [ ]:
# Dataset params
NROWS_TRAIN     = 1000
NROWS_VAL       = 1000

# Tokenisation - fixed
NGRAM_STRIDE    = [(3,1),(5,3),(7,5)]  #(ngram,stride) combinations for tokenisation

# Tokenisation - variable
MAX_VOCAB       = [64, 1024, 16438]

## 1.2 Data Initialisation
This process assumes data has already been read and processed into `.csv` format via the `process_fasta` script.
Data is read from `human_genome_data_fa.csv` in chunks to reduce data in memory. This can then be iterated with `df.next()`.

In [ ]:
df_iter = pd.read_csv(HUMAN/'human_genome_data_fa.csv', chunksize=NROWS_TRAIN+NROWS_VAL)
df = next(df_iter)

# set val to be first 20k rows
df_tr = df[:NROWS_TRAIN]
df_va = df[NROWS_TRAIN:NROWS_TRAIN+NROWS_VAL]

---
# 2. Tokenisation
## 2.1 Fixed length tokenisation
- Technique of original author was to iterate over entire genome sequence with for loops.
- We need to vectorise this.

In [ ]:
# from tools.tokenizers import seq_tokenizer, vec_tokenizer

def seq_tokenizer(t, ngram, stride):
    import time
    t = t.upper()
    if ngram == 1:
        toks = list(t)
    else:
        start = time.time()
        toks = [t[i:i+ngram] for i in range(0, len(t), stride) if len(t[i:i+ngram]) == ngram]

    if len(toks[-1]) < ngram:
        toks = toks[:-1]

    return toks,time.time() - start

def vec_tokenizer(a, ngram=3, stride=1, padnum=0):
    import time
    start = time.time()
    a = np.array(list(a)).astype(object)
    n = a.strides[0]
    nrows = ((a.size)//stride)
    strided = np.lib.stride_tricks.as_strided
    out = strided(a[(ngram-1):-(stride-1)], shape=(nrows,ngram), strides=(stride*n,-n))[:,::-1] 
    out = out[ngram-1:,:]
    toks = list(out[:,0] + out[:,1] + out[:,2])
    return toks,time.time() - start

# Test how well the vectorisation works
def test_tokenizers():
    seq_=[]; vec_=[]
    for i in [1,10,100,1000,10000]:
        test = ''.join([''.join(row) for row in df.head(i).values[:,0]])
        toks ,time = seq_tokenizer(test, 3, 1);seq_.append(time)
        toks_,time_ = vec_tokenizer(test, 3, 1);vec_.append(time_)
    
    fig,ax = plt.subplots(figsize=(12,6))

    stats=pd.DataFrame({'seq':seq_,'vec':vec_})
    stats.plot(ax=ax)
    ax.set_title('Sequential vs Vectorised Tokenisation')
    ax.set_ylabel('Time')
    
test_tokenizers()

### Vectorised technique for faster fixed Length tokenisation

In [ ]:
import time

class FixedLengthGenomicTokenizer(BaseTokenizer):
    """
    Fixed length tokenisation for DNA.
    """
    def __init__(self, lang='en', ngram=5, stride=2):
        self.lang = lang
        self.ngram = ngram
        self.stride = stride
        
    def tokenizer(self, t):
        t = t.upper()
        if self.ngram == 1:
            toks = list(t)
        else:
            toks = [t[i:i+self.ngram] for i in range(0, len(t), self.stride) if len(t[i:i+self.ngram]) == self.ngram]
        if len(toks[-1]) < self.ngram:
            toks = toks[:-1]
        return toks
    
    def add_special_cases(self, toks):
        pass
    
def generate_fixed_vocab(df_train, df_val, ngram=3, stride=1):
    """Create fixed length tokenizer, initialise databunch and return vocabulary."""
    
    # initialise tokeniser
    tok = Tokenizer(partial(GenomicTokenizer, ngram=ngram, stride=stride), 
                    n_cpus=40,
                    pre_rules=[],
                    post_rules=[],
                    special_cases=[])
    
    data = GenomicTextLMDataBunch.from_df(HUMAN, df_train, df_val, bs=NROWS_TRAIN, tokenizer=tok, 
                              chunksize=NROWS_TRAIN, text_cols=0, label_cols=1, max_vocab=((4**ngram)+1))

    # Save and load vocab
    np.save(HUMAN / 'fixed_vocab_{}m{}s.npy'.format(ngram,stride), data.vocab.itos)
    return data.vocab.itos
    
vocabs = []
for ngram_stride in NGRAM_STRIDE:
    vocabs.append(generate_fixed_vocab(df_tr, df_va, *ngram_stride))

In [ ]:
def generate_variable_vocab(df_train, df_val, size=128):
    """Create variable length vocabulary using SentencePiece tokenisation.
    """
    sp_proc = SPProcessor(char_coverage=1, 
                          vocab_sz=size,
                          n_cpus = 40)
    
    data = GenomicTextLMDataBunch.from_df(
        HUMAN, df_train, df_val, bs=NROWS_TRAIN, processor=sp_proc,
        chunksize=NROWS_TRAIN, text_cols=0, label_cols=1, max_vocab=size
    )
    
    # Save and load vocab
    np.save(HUMAN / 'variable_vocab_{}tok.npy'.format(size), data.vocab.itos)
    return data.vocab.itos
    
for max_vocab in MAX_VOCAB:
    vocabs.append(generate_variable_vocab(df_tr, df_va, size=max_vocab))

In [ ]:
pd.DataFrame(vocabs).T

### Variable length tokenisation

In [ ]:
len(df) * len(df.values[0][0])

In [ ]:
SP_MODEL = str(THESIS / "models/genome10M/spiece.model")

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load(str(THESIS / "models/genome10M/spiece.model"))

# encode: text => id
print(sp.encode_as_pieces('GTGAAGGAGCAGGGGCTCCACGTCTGGCGACAACCAGGGAA'))
print(sp.encode_as_ids('GTGAAGGAGCAGGGGCTCCACGTCTGGCGACAACCAGGGAA'))

# decode: id => text
print(sp.decode_pieces(['▁', 'GTGA', 'AGGAG', 'CAGG', 'GGCT', 'CCAC', 'GTC', 'TGGC', 'GACA', 'ACC', 'AGGGAA']))
print(sp.decode_ids([45, 249, 203, 57, 199, 87, 72, 181, 251, 102, 520]))

In [ ]:
def plot_tok_dist():
    
    fig, axs = plt.subplots(nrows=2, figsize=(12,8))
    
    pd.Series(tokens)\
        .value_counts().head(20).plot.bar(ax=axs[0])
    axs[0].set_title('Most common tokens')
    axs[0].set_ylabel('Count')
    
    pd.Series([len(token) for token in tokens])\
        .value_counts().sort_index().plot.bar(ax=axs[1])
    axs[1].set_title('Token Length Distribution')
    axs[1].set_ylabel('Count')
    
    plt.tight_layout()
    
plot_tok_dist()

# Experimental Setup
We propose to explore model performance across different fixed and variable token parameters.
For fixed length tokenisation, the parameters of interest are the number of base pairs per token, also known as `ngram`.
Additionally, the number of base pairs between the start of 1 token and the next is known as `stride`. As an example, take the sequence of base pairs:
$$\text{TCTGGCGACAACCAGGGA}$$

Using fixed length tokenisations of size **_3_** and stride **_0_**, we have the following outputs:
$$\text{[TCT],[GGC],[GAC],[AAC],[CAG],[GGA]}$$

For parameters: `{size:3, stride:1}`
$$\text{[TCT],[],[],[GGC],[],[],[GAC],[],[],[AAC],[],[],[CAG],[],[],[GGA]}$$

For parameters: `{size:5, stride:3}`
$$\text{[TCT],[GGC],[GAC],[AAC],[CAG],[GGA]}$$

For parameters: `{size:7, stride:5}`
$$\text{[TCT],[GGC],[GAC],[AAC],[CAG],[GGA]}$$


In [ ]:
 # initialise tokeniser
tok = Tokenizer(partial(GenomicTokenizer, ngram=ngram, stride=1), 
                n_cpus=64, 
                pre_rules=[], 
                post_rules=[], 
                special_cases=['xxpad'])

# Fixed length tokenisation - 3m1
data_fixed = GenomicTextLMDataBunch.from_df(
    HUMAN, df[20000:40000], df_val, tokenizer=tok,
    bs=3200, vocab=model_vocab, max_vocab=len(voc),
    chunksize=10000, text_cols=0, label_cols=1)

# SentencePiece Tokenisation
sp_proc = SPProcessor(char_coverage=1, 
                      vocab_sz=size,
                      n_cpus = 40)

data_var = data = GenomicTextLMDataBunch.from_df(
    HUMAN, df[20000:], df_val, bs=800, processor=sp_proc, 
    chunksize=10000, text_cols=0, label_cols=1, max_vocab=4^ngram+1)

In [ ]:
# Load vocab
ngram       = 3
stride      = 1
voc         = np.load(HUMAN / 'human_vocab_3m1s.npy')
model_vocab = GenomicVocab(voc)

print("Loaded vocabulary. Number of tokens: {}. Mean token length: {}." \
      .format(len(voc), len(voc)))

### Comparison

In [ ]:
# Load vocab
ngram       = 3
stride      = 1
voc         = np.load(HUMAN / 'human_vocab_3m1s.npy')
model_vocab = GenomicVocab(voc)

print("Loaded vocabulary. Number of tokens: {}. Mean token length: {}.".format(len(voc), len(voc)))

In [ ]:
 # initialise tokeniser
tok = Tokenizer(partial(GenomicTokenizer, ngram=ngram, stride=1), 
                n_cpus=64, 
                pre_rules=[], 
                post_rules=[], 
                special_cases=['xxpad'])

data = GenomicTextLMDataBunch.from_df(
    HUMAN, df[20000:40000], df_val, tokenizer=tok,
    bs=3200, vocab=model_vocab, max_vocab=len(voc),
    chunksize=10000, text_cols=0, label_cols=1)

In [ ]:
data.batch_size

---
# Model initialisation

In [ ]:
config = dict(emb_sz=400, 
              n_hid=1150, 
              n_layers=3, 
              pad_token=0, 
              qrnn=False, 
              output_p=0.25, 
              hidden_p=0.1, 
              input_p=0.2, 
              embed_p=0.02, 
              weight_p=0.15, 
              tie_weights=True, 
              out_bias=True)

drop_mult=0.3

learn = get_model_LM(data, drop_mult, config)
learn = learn.to_fp16(dynamic=True); # convert model weights to 16-bit float

learn.load('human_3m1s')

In [ ]:
learn.model

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, 5e-3, moms=(0.8, 0.7))

In [ ]:
learn.save('human_3m1s')
learn.save_encoder('human_3m1s_enc')

---


In [ ]:
%load_ext tensorboard

In [ ]:
import fastai
fastai.__version__

In [ ]:
import tensorboardX

In [ ]:
from fastai.callbacks.tensorboard import LearnerTensorboardWriter
from fastai.callbacks.csv_logger import CSVLogger
from pathlib import Path

project_id = 'exp1'
tboard_path = Path('./logs/' + project_id)
learn.callback_fns.append(partial(LearnerTensorboardWriter, 
                                    base_dir=tboard_path, 
                                    name='run1'))

learn.callback_fns.append(partial(CSVLogger, append=True))

In [ ]:
learn.fit_one_cycle(1, 5e-3, moms=(0.8, 0.7))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.losses

---

In [ ]:
data = GenomicTextLMDataBunch.from_df(path, df[20000:40000], df_val, bs=3000, tokenizer=tok, vocab=model_vocab, max_vocab=80000,
                              chunksize=10000, text_cols=0, label_cols=1)

config = dict(emb_sz=400, 
              n_hid=1150, 
              n_layers=3, 
              pad_token=0, 
              qrnn=False, 
              output_p=0.25, 
              hidden_p=0.1, 
              input_p=0.2, 
              embed_p=0.02, 
              weight_p=0.15, 
              tie_weights=True, 
              out_bias=True)

drop_mult=0.3

learn = get_model_LM(data, drop_mult, config)
learn = learn.to_fp16(dynamic=True);

learn.fit_one_cycle(2, 5e-3, moms=(0.8, 0.7))

In [ ]:
voc = np.load(path/'human_vocab_3m1s.npy')
model_vocab = GenomicVocab(voc)

In [ ]:
count = 0
lr = 5e-3
for df in df_iter:
    data = GenomicTextLMDataBunch.from_df(path, df, df_val, bs=800, tokenizer=tok, vocab=model_vocab, max_vocab=80000,
                                  chunksize=20000, text_cols=0, label_cols=1)
    learn.data = data                        
    lr_iter = lr/1.5**count
    print(f'Learning Rate: {lr_iter}')
    learn.fit_one_cycle(1, lr, moms=(0.8,0.7))
    count += 1

In [ ]:
learn.save('human_3m1s2')
learn.save_encoder('human_3m1s_enc2')

In [ ]:
learn.load('human_3m1s2');

In [ ]:
learn = learn.to_fp32();

In [ ]:
learn.save('human_3m1s2_fp32')